In [1]:
import os
from datasets import Dataset
import h5py
import re
from transformers import T5Tokenizer

In [2]:
file_name = "_dummy"

with h5py.File(f"../tmp/data/pssm/pssm_data{file_name}.h5", "r") as f:
    print(len(list(f.keys())))
    print(list(f["1k87A01"].attrs))
    print()
    print(f["1k87A01"].attrs['name'])
    print(f["1k87A01"].attrs['sequence'])
    print(list(f["1k87A01"].keys()))
    print(type(f["1k87A01"]["320_0"][:]))
    print(f["1k87A01"]["320_0"][:].shape)
    print(f["1k87A01"]["320_0"][:])
    print()
    # for x in range(1000):
    # for x in list(f["1k87A01"].keys()):
    #     # asdf = f["1k87A01"][x][:]
    #     print(f["1k87A01"][x][:].shape)

5398
['name', 'sequence', 'structure']

1k87A01
PQSVSRAAITAAYRRPETEAVSMLLEQARLPQPVAEQAXKLAYQLADKLRNQ
['320_0', '320_1', '320_2', '320_3', '320_4', '348_0', '348_1', '348_2', '348_3', '348_4', '379_0', '379_1', '379_2', '379_3', '379_4', '413_0', '413_1', '413_2', '413_3', '413_4', '450_0', '450_1', '450_2', '450_3', '450_4']
<class 'numpy.ndarray'>
(52, 20)
[[0.0292 0.0064 0.753  ... 0.0316 0.0065 0.0061]
 [0.0462 0.0585 0.1665 ... 0.3268 0.0144 0.0022]
 [0.0992 0.026  0.2399 ... 0.1783 0.0061 0.0034]
 ...
 [0.0074 0.0151 0.1027 ... 0.5476 0.0015 0.0109]
 [0.0128 0.0127 0.2188 ... 0.3651 0.0028 0.0045]
 [0.0292 0.0064 0.753  ... 0.0316 0.0065 0.0061]]



In [3]:
tokenizer = T5Tokenizer.from_pretrained(
        pretrained_model_name_or_path='Rostlab/prot_t5_xl_uniref50',
        do_lower_case=False,
        use_fast=True,
        legacy=False,
    )

def gen():
    with h5py.File(f"../tmp/data/pssm/pssm_data{file_name}.h5", "r") as f:
        for protein_id in f.keys():
            for trajectory_pssm in f[protein_id].keys():
                sequence_processed = " ".join(list(re.sub(r"[UZOB]", "X", f[protein_id].attrs['sequence'])))
                sequence_tokenized = tokenizer(text=sequence_processed, padding=False, truncation=False)
                input_ids = sequence_tokenized['input_ids']
                attention_mask = sequence_tokenized['attention_mask']
                yield {"name": f[protein_id].attrs['name'] + "_" + trajectory_pssm, "sequence": f[protein_id].attrs['sequence'], "sequence_processed":  sequence_processed, "input_ids": input_ids, "attention_mask": attention_mask, "pssm_features": f[protein_id][trajectory_pssm][:]}
ds = Dataset.from_generator(gen)

ds.save_to_disk(f"../tmp/data/pssm/pssm_dataset{file_name}")

Generating train split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/134950 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk
ds = load_from_disk(f"../tmp/data/pssm/pssm_dataset{file_name}")

print("Dataset created with", len(ds), "examples")
print("\nDataset:", ds)
print()
print(ds[0])
print(type(ds[0]['name']))
print(type(ds[0]['sequence']))
print(type(ds[0]['sequence_processed']))
print(type(ds[0]['pssm_features']))

print("Dataset saved successfully")

Dataset created with 134950 examples

Dataset: Dataset({
    features: ['name', 'sequence', 'sequence_processed', 'input_ids', 'attention_mask', 'pssm_features'],
    num_rows: 134950
})

{'name': '12asA00_320_0', 'sequence': 'AYIAKQRQISFVKSXFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVXSLAKWKRQTLGQXDFSAGEGLYTXMKALRPDEDRLSPLXSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIXFVXSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGXRXDVRAPDYDDWSTPSELGXAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKXQLALTGDEDRLELEWXQALLRGEMPQTIGGGIGQSRLTMLLLQLPXIGQVQAGVWPAAVRESVPSLL', 'sequence_processed': 'A Y I A K Q R Q I S F V K S X F S R Q L E E R L G L I E V Q A P I L S R V G D G T Q D N L S G A E K A V Q V K V K A L P D A Q F E V V X S L A K W K R Q T L G Q X D F S A G E G L Y T X M K A L R P D E D R L S P L X S V Y V D Q W D W E R V M G D G E R Q F S T L K S T V E A I W A G I K A T E A A V S E E F G L A P F L P D Q I X F V X S Q E L L S R Y P D L D A K G R E R A I A K D L G A V F L V G I G G K L S D G X R X D V 

In [5]:
ds = ds.filter(lambda x: x['name'].endswith('_0'))
print(ds)
ds.save_to_disk(f"../tmp/data/pssm/pssm_dataset{file_name}_0_only")

Filter:   0%|          | 0/134950 [00:00<?, ? examples/s]

Dataset({
    features: ['name', 'sequence', 'sequence_processed', 'input_ids', 'attention_mask', 'pssm_features'],
    num_rows: 26990
})


Saving the dataset (0/2 shards):   0%|          | 0/26990 [00:00<?, ? examples/s]